In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import geopandas as gpd
import statsmodels.formula.api as smf
import seaborn as sns

In [85]:
nbhd_transit = pd.read_csv('Overall_nbhd_transit_eq.csv', index_col=0)

In [86]:

zhvi_df = pd.read_csv("sd_zhvi_2000_to_2025_nhbd.csv")
zhvi_df.drop(['RegionType', 'StateName', 'State', 'City', 'Metro', 'CountyName'], axis=1, inplace=True)
zhvi_cols = zhvi_df.columns[3:]
obs_started = zhvi_df[zhvi_cols].notnull().cumsum(axis=1) > 0
incomp_obs = zhvi_df[zhvi_cols].isnull() & obs_started
nbhds_incomp = incomp_obs.any(axis=1)
problem_rows = zhvi_df.loc[nbhds_incomp]
rows_clean = ~incomp_obs.any(axis=1)
clean_rows_df = zhvi_df.loc[rows_clean]
def quarter_nonnull(row):
    first_valid = row.first_valid_index()
    if first_valid is None:
        return False

    start = row.index.get_loc(first_valid) + 1
    tb_complete = row.iloc[start:]

    for i in range(0, len(tb_complete), 3):
        group = tb_complete.iloc[i:i + 3]
        if group.notnull().sum() == 0:
            return False 
    return True

mask_valids = zhvi_df[zhvi_cols].apply(quarter_nonnull, axis=1)
clean_zhvi = zhvi_df.loc[mask_valids]
clean_zhvi
clean_zhvi['RegionName'] = clean_zhvi['RegionName'].str.upper()

We need to standardize neighborhood names

In [70]:
clean_zhvi['RegionName'] = clean_zhvi['RegionName'].replace({'JOMACHA-LOMITA': 'JAMACHA LOMITA', 'MT. HOPE': 'MT HOPE', 'BANKERS HILL': 'PARK WEST', 'BIRD LAND': 'BIRDLAND', 'RIDGEVIEW-WEBSTER': 'RIDGEVIEW/WEBSTER', 'CORTEZ HILL': 'CORTEZ', 'RANCHO ENCANTADO': 'RANCHO ENCANTADA', 'GASLAMP QUARTER': 'GASLAMP', 'AZALEA-HOLLYWOOD PARK': 'AZALEA/HOLLYWOOD PARK', 'ROSEVILLE - FLEET RIDGE': 'ROSEVILLE / FLEET RIDGE'})

NEIGHBORHOOD - BROADWAY HEIGHTS NOT COUNTED? Lack of Transit & Lack of Home Ownership and Pricing Data - African-American community in San Diego

In [71]:
for col in clean_zhvi.columns:
    if str(col)[0].isdigit() == True:
        year = int(str(col)[:4])
        month = int(str(col)[5:7])
        if year < 2012:
            clean_zhvi = clean_zhvi.drop(col, axis=1)
        elif year == 2012 and month < 6:
            clean_zhvi = clean_zhvi.drop(col, axis=1)
    

In [72]:
diff_1 = set(list(nbhd_transit['neighborhood'])) -  set(list(clean_zhvi['RegionName']))
diff_1 = list(diff_1)
nbhd_transit = nbhd_transit[~nbhd_transit['neighborhood'].isin(diff_1)]

In [73]:
yymm_cols = [col for col in nbhd_transit.columns if str(col)[0].isdigit()]
transit_long = pd.melt(nbhd_transit, id_vars=['neighborhood', 'geometry', 'no_change', 'no_transit'], value_vars=yymm_cols)
transit_long = transit_long.rename(columns={'variable': 'yymm', 'value': 'trans_score'})

In [74]:
def rename_col(col):
    col = str(col)
    if col[0].isdigit() == True:
        return (col[2:4] + col[5:7])
    else:
        return col

clean_zhvi = clean_zhvi.rename(columns=lambda col: rename_col(col))

In [75]:
yymm_cols = [col for col in clean_zhvi.columns if str(col)[0].isdigit()]

zhvi_long = pd.melt(clean_zhvi, id_vars=['RegionID', 'SizeRank', 'RegionName'], value_vars=yymm_cols)
zhvi_long = zhvi_long.rename(columns={'RegionName':'neighborhood', 'variable':'yymm', 'value':'ZHVI'})

In [76]:
zhvi_transit = pd.merge(zhvi_long[['neighborhood', 'yymm', 'ZHVI']], transit_long[['neighborhood', 'yymm', 'trans_score']], on=['neighborhood', 'yymm'], how='left')
zhvi_transit = zhvi_transit.sort_values(['neighborhood', 'yymm'])
zhvi_transit['trans_score'] = zhvi_transit.groupby('neighborhood')['trans_score'].ffill()

In [77]:
const_zhvi = clean_zhvi[['RegionName', 'SizeRank', 'RegionID']].rename(columns={'RegionName':'neighborhood'})
const_trans = nbhd_transit[['neighborhood', 'geometry', 'no_change', 'no_transit']]

zhvi_transit = pd.merge(zhvi_transit, const_zhvi, on='neighborhood', how='left')

zhvi_transit = pd.merge(zhvi_transit, const_trans, on='neighborhood', how='left')


In [78]:
#zhvi_transit.to_csv('housing_transit_overall_merged.csv', index = True)

In [79]:
zhvi_transit['score_diff'] = zhvi_transit.groupby('neighborhood')['trans_score'].diff()
score_changes = zhvi_transit[zhvi_transit['score_diff'].notna() & (zhvi_transit['score_diff'] != 0)]

In [80]:
def shift_yymm(yymm, months):
    year = yymm // 100
    month = yymm % 100
    total_months = year * 12 + month + months
    new_year = total_months // 12
    new_month = total_months % 12
    if new_month == 0:
        new_year -= 1
        new_month = 12
    return new_year * 100 + new_month

In [81]:
results = []
min_yymm = 1206
max_yymm = 2503

for idx, row in score_changes.iterrows():
    treated_nhood = row['neighborhood']
    treatment_date = int(row['yymm'])
    change_direction = np.sign(row['score_diff'])

    # Define date bounds
    min_yymm = 1206
    max_yymm = 2503

    # Get all dates for the treated neighborhood
    nhood_df = zhvi_transit[zhvi_transit['neighborhood'] == treated_nhood].copy()
    nhood_df = nhood_df.sort_values('yymm')

    # Find the previous date when score changed
    previous_change = nhood_df[(nhood_df['yymm'].astype(int) < treatment_date) & (nhood_df['score_diff'] != 0)]
    if not previous_change.empty:
        min_pre_date = int(previous_change.iloc[-1]['yymm'])
    else:
        min_pre_date = shift_yymm(treatment_date, -6)

    # Find the next date when score changes again
    next_change = nhood_df[(nhood_df['yymm'].astype(int) > treatment_date) & (nhood_df['score_diff'] != 0)]
    if not next_change.empty:
        max_post_date = int(next_change.iloc[0]['yymm'])
    else:
        max_post_date = shift_yymm(treatment_date, 6)

    # Ensure bounds are within overall date limits
    max_post_date = min(max_post_date, max_yymm)
    min_pre_date = max(min_pre_date, min_yymm)

    # Get analysis period between min_pre_date and max_post_date
    analysis_period = zhvi_transit[(zhvi_transit['yymm'].astype(int) >= min_pre_date) & 
                                   (zhvi_transit['yymm'].astype(int) <= max_post_date)].copy()

    # Get pre-treatment transit score for treated neighborhood
    pre_score = row['trans_score'] - row['score_diff']

    # Identify candidate control neighborhoods with no change in score during this period
    group_scores = analysis_period.groupby(['neighborhood'])['trans_score'].nunique().reset_index()
    static_nhoods = group_scores[group_scores['trans_score'] == 1]['neighborhood'].tolist()

    # Get score at min_pre_date for candidate control neighborhoods
    control_scores = zhvi_transit[(zhvi_transit['neighborhood'].isin(static_nhoods)) & 
                                  (zhvi_transit['yymm'].astype(int) == min_pre_date)].copy()

    # Filter based on direction of change
    if change_direction > 0:
        candidate_controls = control_scores[control_scores['trans_score'] <= pre_score]['neighborhood'].tolist()
    else:
        candidate_controls = control_scores[control_scores['trans_score'] >= pre_score]['neighborhood'].tolist()

    # Subset to treated + control neighborhoods
    selected_nhoods = [treated_nhood] + candidate_controls
    sub_df = analysis_period[analysis_period['neighborhood'].isin(selected_nhoods)].copy()

    # Mark treatment group
    sub_df['treated'] = (sub_df['neighborhood'] == treated_nhood).astype(int)

    # Mark post-treatment period
    sub_df['post'] = (sub_df['yymm'].astype(int) >= treatment_date).astype(int)

    # Create interaction term
    sub_df['treated_post'] = sub_df['treated'] * sub_df['post']

    sub_df['trend'] = sub_df.groupby('neighborhood')['ZHVI'].transform(lambda x: x.shift(1) - x.shift(2))

    # Skip if insufficient data
    if sub_df['treated'].sum() < 3 or sub_df.shape[0] < 20 or len(candidate_controls) == 0:
        continue

    try:
        # Run DiD regression with fixed effects, controlling for ZHVI level and trend
        model = smf.wls('ZHVI ~ treated + post + treated_post + trend + C(neighborhood) + C(yymm)', 
                        data=sub_df, 
                        weights=(sub_df['SizeRank'].max() - sub_df['SizeRank'] + 1)).fit()

        results.append({
            'neighborhood': treated_nhood,
            'treatment_yymm': treatment_date,
            'coef': model.params.get('treated_post', np.nan),
            'pval': model.pvalues.get('treated_post', np.nan),
            'n_obs': sub_df.shape[0],
            'direction': 'increase' if change_direction > 0 else 'decrease'
        })
    except Exception as e:
        print(f"Error in regression for neighborhood {treated_nhood}: {e}")

# STEP 4: Combine results
results_df = pd.DataFrame(results)

# Print average treatment effects by direction
print("Average Treatment Effect (ATE) - Increases:", results_df[results_df['direction'] == 'increase']['coef'].mean())
print("Average Treatment Effect (ATE) - Decreases:", results_df[results_df['direction'] == 'decrease']['coef'].mean())
print(results_df[['neighborhood', 'treatment_yymm', 'coef', 'pval', 'n_obs', 'direction']])
    

Average Treatment Effect (ATE) - Increases: -41393.55303301876
Average Treatment Effect (ATE) - Decreases: -49665.58967631425
           neighborhood  treatment_yymm           coef          pval  n_obs  \
0          BARRIO LOGAN            2106  -36822.250063  2.327788e-03   8740   
1                BAY HO            2111   38260.061307  2.579565e-03   8400   
2              BAY PARK            2111   66660.764684  1.313157e-07   8400   
3                CASTLE            2106  -41471.714938  7.389108e-04   8050   
4        CHEROKEE POINT            2106  -49040.978903  1.744049e-04   8050   
5          CHOLLAS VIEW            2106  -39115.968750  4.004664e-03   8740   
6          COLLEGE EAST            2106    8233.395495  4.872755e-01   8740   
7          COLLEGE WEST            2106   20117.320907  9.752218e-02   8740   
8         CORE-COLUMBIA            2106 -135528.705303  6.862752e-28   8740   
9              CORRIDOR            2106  -43832.083876  4.200633e-04   8050   
10   

In [98]:
zhvi_transit[zhvi_transit['neighborhood'] == 'SKYLINE']

,neighborhood,yymm,ZHVI,trans_score,SizeRank,RegionID,geometry,no_change,no_transit,score_diff
14938,SKYLINE,1206,217327.3360,2.0,2461,275386,"POLYGON ((6324450.5 1837023.1252499968, 632445...",0,0,NaN
14939,SKYLINE,1206,217327.3360,2.0,2461,275386,POLYGON ((6319511.503000006 1839815.8682499975...,0,0,0.0
14940,SKYLINE,1206,217327.3360,2.0,2461,275386,"POLYGON ((6324450.5 1837023.1252499968, 632445...",0,0,0.0
14941,SKYLINE,1206,217327.3360,2.0,2461,275386,POLYGON ((6319511.503000006 1839815.8682499975...,0,0,0.0
14942,SKYLINE,1207,218859.4585,2.0,2461,275386,"POLYGON ((6324450.5 1837023.1252499968, 632445...",0,0,0.0
...,...,...,...,...,...,...,...,...,...,...
15327,SKYLINE,2501,739608.6822,6.0,2461,275386,POLYGON ((6319511.503000006 1839815.8682499975...,0,0,0.0
15328,SKYLINE,2502,736730.1364,6.0,2461,275386,"POLYGON ((6324450.5 1837023.1252499968, 632445...",0,0,0.0
15329,SKYLINE,2502,736730.1364,6.0,2461,275386,POLYGON ((6319511.503000006 1839815.8682499975...,0,0,0.0
15330,SKYLINE,2503,732126.3977,6.0,2461,275386,"POLYGON ((6324450.5 1837023.1252499968, 632445...",0,0,0.0
